### Machine Learning Model Training and Evaluation:

In [1]:
import numpy as np
import pandas as pd
import kerastuner as kt

C:\Users\koeli\AppData\Local\Temp\ipykernel_14784\3930637375.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [2]:
# Importing Transformed Dataset:
df = pd.read_csv('../Dataset/Loan_default_transformed.csv')
df.sample(4)

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,...,HasMortgage,HasDependents,HasCoSigner,Default,MaritalStatus_Married,MaritalStatus_Single,LoanPurpose_Business,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other
102139,0.076923,0.365425,0.252328,0.142077,0.823529,1.0,0.099565,1.00,0.70,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
107850,0.461538,0.676064,0.149650,0.881603,0.529412,1.0,0.055217,0.75,0.17,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
177855,0.769231,0.837651,0.505161,0.992714,0.882353,4.0,0.107826,0.50,0.41,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
120623,0.538462,0.331765,0.961730,0.819672,0.042017,3.0,0.428696,1.00,0.28,3.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
df['Default'].value_counts()

Default
0.0    225694
1.0     29653
Name: count, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Age                    255347 non-null  float64
 1   Income                 255347 non-null  float64
 2   LoanAmount             255347 non-null  float64
 3   CreditScore            255347 non-null  float64
 4   MonthsEmployed         255347 non-null  float64
 5   NumCreditLines         255347 non-null  float64
 6   InterestRate           255347 non-null  float64
 7   LoanTerm               255347 non-null  float64
 8   DTIRatio               255347 non-null  float64
 9   Education              255347 non-null  float64
 10  EmploymentType         255347 non-null  float64
 11  HasMortgage            255347 non-null  float64
 12  HasDependents          255347 non-null  float64
 13  HasCoSigner            255347 non-null  float64
 14  Default                255347 non-nu

In [5]:
df[df.isnull().any(axis=1)]

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,...,HasMortgage,HasDependents,HasCoSigner,Default,MaritalStatus_Married,MaritalStatus_Single,LoanPurpose_Business,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other


In [6]:
# Using downsampling
from sklearn.utils import resample

## Downsampling to reduce dataset size:
df_majority = df[df['Default'] == 0]
df_minority = df[df['Default'] == 1]

df_majority = resample(df_majority, n_samples=len(df_minority),random_state=42)

df = pd.concat([df_majority, df_minority])

In [7]:
## Splitting the Dataset:
X = df.drop('Default',axis=1)
y = df[['Default']]

from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47444, 20), (11862, 20), (47444, 1), (11862, 1))

#### Firstly, we will balance our classes using smote:

In [8]:
# from imblearn.over_sampling import SMOTE

# # Apply SMOTE to balance the dataset
# smote = SMOTE(sampling_strategy=0.8, random_state=42)  # Make class 1 = 50% of class 0
# X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train.values.ravel())

# print("Before SMOTE:", y_train.value_counts())  # Check class distribution
# print("After SMOTE:", pd.Series(y_train_balanced).value_counts())

#### Create an Evaluate Function to give all metrics after model Training:

In [9]:
## All classification models:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

models = {
   'Logistic Regression': LogisticRegression(),
   'Decision Tree': DecisionTreeClassifier(),
   # 'SVM': SVC(),
   'Guassian': GaussianNB(),
   # 'KNN': KNeighborsClassifier(),
   'AdaBoost': AdaBoostClassifier(),
   'Gradient Boost': GradientBoostingClassifier(),
   'XGBoost': XGBClassifier()
}

In [10]:
from sklearn.metrics import accuracy_score

model_list = []
acc_list = []

## Fit each model, predict and store it with model's accuracy
for name, model in models.items():
   # Fitting model
   model.fit(X_train,y_train.values.ravel())
   # Prediction
   y_pred = model.predict(X_test)
   
   acc = accuracy_score(y_test,y_pred)
   model_list.append(name)
   acc_list.append(acc)
   
   print(f'Model performance of {name} for Test set')
   print("- Accuracy score: {:.4f}".format(acc))
   
   print('='*35)
   print('\n')

Model performance of Logistic Regression for Test set
- Accuracy score: 0.6902


Model performance of Decision Tree for Test set
- Accuracy score: 0.6132


Model performance of Guassian for Test set
- Accuracy score: 0.6850


Model performance of AdaBoost for Test set
- Accuracy score: 0.6870


Model performance of Gradient Boost for Test set
- Accuracy score: 0.6943


Model performance of XGBoost for Test set
- Accuracy score: 0.6801




XGboost and Logistic Regression came out to best models

In [11]:
## Fitting Logistic Regression to its best:
LR = LogisticRegression()

param_grid = [
   {
      'penalty': ['l1', 'l2'],
      'solver': ['liblinear'],
      'max_iter': [100, 200, 500]
   },  {
      'penalty': ['l2'],
      'solver': ['lbfgs', 'newton-cg', 'sag'],  # solvers that only support l2
      'max_iter': [100, 200, 500]
   },  {
      'penalty': ['l1','l2','elasticnet'],
      'solver': ['saga'], 
      'max_iter': [100, 200, 500],
      'l1_ratio': [0.1, 0.5, 0.9],
   }
]

In [12]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

clf = GridSearchCV(LR, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1, error_score='raise')
best_LR = clf.fit(X_train, y_train.values.ravel())

Fitting 3 folds for each of 42 candidates, totalling 126 fits


In [13]:
Best_LR = best_LR.best_params_

# Get the best parameters and score
print("Best parameters found: ", Best_LR)

pred_LR = best_LR.predict(X_test)
print("Best Accuracy from Logistic Regression: ", accuracy_score(y_test, pred_LR))

Best parameters found:  {'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best Accuracy from Logistic Regression:  0.6901871522508852


In [14]:
## Parameters for XGBoost
Param_dict = {
   'max_depth': [3, 4, 5, 6, 8, 10],
   'min_child_weight': [3, 5, 7],
   'gamma': [0, 0.1, 0.2, 0.3, 0.4],
   'max_iter': [100, 200, 500],
}

In [15]:
from sklearn.model_selection import RandomizedSearchCV

XG = XGBClassifier()

clf = RandomizedSearchCV(XG, param_distributions=Param_dict, n_iter=30, cv=3, verbose=2, n_jobs=-1)
best_XG = clf.fit(X_train, y_train.values.reshape(-1,1))

Fitting 3 folds for each of 30 candidates, totalling 90 fits


c:\D Drive\coding\Python-projects\Loan_Prediction\VirtualEnv\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:09:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_iter" } are not used.

  warnings.warn(smsg, UserWarning)


In [16]:
Best_XG = best_XG.best_params_

# Get the best parameters and score
print("Best parameters found: ", Best_XG)

pred_XG = best_XG.predict(X_test)
print("Best Accuracy XGBoost: ", accuracy_score(y_test, pred_XG))

Best parameters found:  {'min_child_weight': 3, 'max_iter': 200, 'max_depth': 3, 'gamma': 0.3}
Best Accuracy XGBoost:  0.6949081099308717


#### Now, We will Fit a Neural Network for Same problem and compare its performance with Normal Machine Learning:

In [17]:
import tensorflow as tf
from tensorflow import keras as kr

In [18]:
## Creating Neural Network:
model = kr.Sequential() # Network base
model.add(kr.layers.Input(shape=(20,))) # Input layer

## Neural Network
# model.add(kr.layers.Dense(256, activation='relu'))
model.add(kr.layers.Dense(128, activation='relu'))
model.add(kr.layers.Dense(64, activation='relu'))
model.add(kr.layers.Dense(32, activation='relu'))
model.add(kr.layers.Dense(16, activation='relu'))

## Output layer
model.add(kr.layers.Dense(1, activation='sigmoid'))

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,569 (53.00 KB)

 Trainable params: 13,569 (53.00 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], )

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
1483/1483 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.6449 - loss: 0.6238 - val_accuracy: 0.6877 - val_loss: 0.5898
Epoch 2/20
1483/1483 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6808 - loss: 0.5974 - val_accuracy: 0.6886 - val_loss: 0.5896
Epoch 3/20
1483/1483 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6802 - loss: 0.5952 - val_accuracy: 0.6938 - val_loss: 0.5867
Epoch 4/20
1483/1483 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6795 - loss: 0.5957 - val_accuracy: 0.6884 - val_loss: 0.5875
Epoch 5/20
1483/1483 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6822 - loss: 0.5897 - val_accuracy: 0.6951 - val_loss: 0.5843
Epoch 6/20
1483/1483 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6851 - loss: 0.5860 - val_accuracy: 0.6906 - val_loss: 0.5919
Epoch 7/20
1483/1483 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6865 - loss: 0.5876 - val_accuracy: 0.6921 - val_loss: 0.5848
Epoch 8/20
1483/1483 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6862 - loss: 0.5885 -

In [21]:
pred_ANN = model.predict(X_test)
pred_ANN_labels = (pred_ANN > 0.5).astype(int)

print("Best Accuracy Neural Network: ", accuracy_score(y_test, pred_ANN_labels))

371/371 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Best Accuracy Neural Network:  0.6852132861237565


Now, we will Tune this Neural Network:

In [22]:
## Function to build mode using various values from hyperparameters
def Build_best_classifier(hp):
   model = kr.Sequential()
   model.add(kr.Input(shape=(20,)))
   
   for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
      ## Adding model layer structure
      model.add(kr.layers.Dense(units=hp.Int("Unit_count",16,128, step=16),
         kernel_initializer='he_normal', kernel_regularizer=kr.regularizers.l2(0.01))
      )
      
      ## Adding activation func and batch normalizer
      model.add(kr.layers.LeakyReLU(negative_slope=0.2))
      model.add(kr.layers.BatchNormalization())
      
      ## Adding a dropout layer
      rate = hp.Float('rate', min_value=0.0, max_value=0.5, step=0.1)
      if hp.Boolean('dropout'):
         model.add(kr.layers.Dropout(rate))
      
   ## Output layer
   model.add(kr.layers.Dense(1, activation='sigmoid'))
   
   model.compile(optimizer=kr.optimizers.Adam(hp.Choice('learning_rate', [0.1, 1e-2, 1e-3])), loss='binary_crossentropy', metrics=['accuracy'])
   
   return model

In [23]:
tuner = kt.RandomSearch(Build_best_classifier, objective='val_accuracy', max_trials=10, directory='project', project_name='Loan_default')

## Fitting model
tuner.search(X_train, y_train, epochs=2, validation_data=(X_test,y_test))

Trial 10 Complete [00h 01m 57s]
val_accuracy: 0.6816725730895996

Best val_accuracy So Far: 0.6843702793121338
Total elapsed time: 00h 14m 09s


> #### Note:  Hyper tuning Neural network or even using ANN dont show any significant improvement. <br> XGBoost gives Best accuracy with less computations.